# Lab 5 Data Structures

Resources
- https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.io.json.json_normalize.html
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html
- https://docs.python.org/2/library/xml.etree.elementtree.html

In [1]:
# import modules
import pandas as pd
import numpy as np

from pandas.io.json import json_normalize #special package in pandas
import json
from lxml import etree
import sqlite3

# Part A: Reading json

In [2]:
# Run this cell to import the following JSON strings
players = '[{"name":"Gazinsky","team":"Russia"},{"name":"Dzyuba","team":"Russia"},{"name":"Lukaku","team":"Belgium"}]'
stadium_data = '{"stadiums":[{"name": "Ekaterinburg Arena","city": "Ekaterinburg"},{"name": "Luzhniki Stadium","city": "Moscow"},{"name": "Nizhny Novgorod Stadium","city": "Nizhny Novgorod"}]}'



In [3]:
# load players string as a json object (using json.loads)

# print the dictionary containing information about the third player only
# HINT: Once the data is in python data structures (i.e. lists/dictionaries) 
# navigate through the nested lists and dictionaries to extract the relevant information
# HINT: Remember, the index starts at 0, so the first player is index 0, the second player is index 1, etc.


playersobj = json.loads(players) 
print(playersobj[2])



{'name': 'Lukaku', 'team': 'Belgium'}


In [4]:
# load stadium_data as a json object
###NEED TO REDO
# print the city of Luzhniki Stadium by navigating through the nested dictionaries and lists (key="city")
stadium_data_obj = json.loads(stadium_data) 
print(stadium_data_obj['stadiums'][1]['city'])

Moscow


In [5]:
# read world cup match data from the worldcup.json file into a json object

# print the date of the first match (key="date")
# HINT: Once the data is in python data structures (i.e. lists/dictionaries) 
# navigate through the nested lists and dictionaries to extract the relevant information
world_cup = open('worldcup.json')
world_cup_obj = json.load(world_cup)

print(world_cup_obj['rounds'][0]['matches'][0]['date'])

2018-06-14


In [6]:
# read world cup team data from the worldcup.teams.json file into json object

# print the name of the first team in the team data set (key="name")
# HINT: Once the data is in python data structures (i.e. lists/dictionaries) 
# navigate through the nested lists and dictionaries to extract the relevant information
world_cup_teams = open('worldcup.teams.json')
world_cup_teams_obj = json.load(world_cup_teams)

print(world_cup_teams_obj['teams'][0]['name'])

Egypt


In [7]:
# manually create a list of dictionaries with the following soccer associations and continents as the keys & values
# The keys should be  "continent" and "association". 

# Europe - Union of European Football Associations 
# Asia - Asian Football Confederation
# Africa - Confederation Africaine de Football 

soccer = [{'continent': 'Europe', 'association':'Union of European Football Associations'}, 
         {'continent': 'Asia', 'association':'Asian Football Confederation'},
         {'continent': 'Africa', 'association':'Confederation Africaine de Football'}]

# Part B: Flattening json

In [8]:
# flatten the players json object created in Part A (second cell) into a data frame with one row per player

# print first few rows of data set
players_df = json_normalize(playersobj)
players_df.head()

/var/folders/3z/502yf7vd36g68b85pr331rqh0000gn/T/ipykernel_47697/3467746262.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  players_df = json_normalize(playersobj)


,name,team
0,Gazinsky,Russia
1,Dzyuba,Russia
2,Lukaku,Belgium


In [9]:
# flatten the stadium data json object created in Part A (third cell) into a data frame with one row per stadium

# print first few rows of data set

stadium_data_df = json_normalize(stadium_data_obj,record_path='stadiums')
stadium_data_df.head()

/var/folders/3z/502yf7vd36g68b85pr331rqh0000gn/T/ipykernel_47697/4252394250.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  stadium_data_df = json_normalize(stadium_data_obj,record_path='stadiums')


,name,city
0,Ekaterinburg Arena,Ekaterinburg
1,Luzhniki Stadium,Moscow
2,Nizhny Novgorod Stadium,Nizhny Novgorod


In [20]:
# flatten the world cup match json object created in Part A (fourth cell)  into a data frame with one row per match
# HINT: The data you want is nested in "rounds", and further in the "matches" key in the world cup match json. You are going to have to get rid of two outer layers.
# HINT: You should include both rounds & matches as nested keys in record path
# (see https://stackoverflow.com/questions/47242845/pandas-io-json-json-normalize-with-very-nested-json)
# e.g. record_path=["level1key",["level2key"]]
# where level1key = "rounds" and level2key = "matches"

# print number of rows

# print first few rows of data set


world_cup_df = json_normalize(world_cup_obj,record_path=["rounds",["matches"]])
print(world_cup_df.shape)
world_cup_df.head()

(64, 23)


/var/folders/3z/502yf7vd36g68b85pr331rqh0000gn/T/ipykernel_47697/2019490135.py:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  world_cup_df = json_normalize(world_cup_obj,record_path=["rounds",["matches"]])


,num,date,time,score1,score2,score1i,score2i,goals1,goals2,group,...,team1.code,team2.name,team2.code,stadium.key,stadium.name,score1et,score2et,score1p,score2p,knockout
0,1,2018-06-14,18:00,5,0,2,0,"[{'name': 'Gazinsky', 'minute': 12, 'score1': ...",[],Group A,...,RUS,Saudi Arabia,KSA,luzhniki,Luzhniki Stadium,NaN,NaN,NaN,NaN,NaN
1,2,2018-06-15,17:00,0,1,0,0,[],"[{'name': 'Giménez', 'minute': 89, 'score1': 0...",Group A,...,EGY,Uruguay,URU,ekaterinburg,Ekaterinburg Arena,NaN,NaN,NaN,NaN,NaN
2,3,2018-06-15,21:00,3,3,2,1,"[{'name': 'Ronaldo', 'minute': 4, 'score1': 1,...","[{'name': 'Costa', 'minute': 24, 'score1': 1, ...",Group B,...,POR,Spain,ESP,fisht,Fisht Stadium,NaN,NaN,NaN,NaN,NaN
3,4,2018-06-15,18:00,0,1,0,0,[],"[{'name': 'Bouhaddouz', 'minute': 90, 'offset'...",Group B,...,MAR,Iran,IRN,saintpetersburg,Saint Petersburg Stadium,NaN,NaN,NaN,NaN,NaN
4,5,2018-06-16,13:00,2,1,0,0,"[{'name': 'Griezmann', 'minute': 58, 'score1':...","[{'name': 'Jedinak', 'minute': 62, 'score1': 1...",Group C,...,FRA,Australia,AUS,kazan,Kazan Arena,NaN,NaN,NaN,NaN,NaN


In [11]:
# flatten the team data json object created in Part A (fifth cell) into a data frame with one row per team

# print number of rows

# print first few rows of data set



world_cup_teams_df = json_normalize(world_cup_teams_obj,record_path=["teams"])
print(world_cup_teams_df.shape[0])
world_cup_teams_df.head()

32


/var/folders/3z/502yf7vd36g68b85pr331rqh0000gn/T/ipykernel_47697/3544503937.py:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  world_cup_teams_df = json_normalize(world_cup_teams_obj,record_path=["teams"])


,name,code,continent,assoc.key,assoc.name,assoc.continental.name,assoc.continental.code
0,Egypt,EGY,Africa,egy,Egyptian Football Association,Confédération Africaine de Football (CAF),CAF
1,Morocco,MAR,Africa,mar,Fédération Royale Marocaine de Football,Confédération Africaine de Football (CAF),CAF
2,Tunisia,TUN,Africa,tun,Fédération Tunisienne de Football,Confédération Africaine de Football (CAF),CAF
3,Senegal,SEN,Africa,sen,Fédération Sénégalaise de Football,Confédération Africaine de Football (CAF),CAF
4,Nigeria,NGA,Africa,nga,Nigeria Football Federation,Confédération Africaine de Football (CAF),CAF


In [12]:
# flatten the soccer association list of dictionaries created in Part A (sixth cell) into a data frame with one row per association

# print number of rows

# print first few rows of data set
soccer_df = json_normalize(soccer)
print(soccer_df.shape[0])
soccer_df.head()

3


/var/folders/3z/502yf7vd36g68b85pr331rqh0000gn/T/ipykernel_47697/2563274309.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  soccer_df = json_normalize(soccer)


,continent,association
0,Europe,Union of European Football Associations
1,Asia,Asian Football Confederation
2,Africa,Confederation Africaine de Football


# Part C: SQL

In [13]:
# create a database and sql connection 
con = sqlite3.connect('m5_database.sqlite')

In [14]:
# save the players data set created in part B as a sql table in your database
players_df.to_sql('players',con,if_exists='replace')

3

In [15]:
# save the stadiums data set created in part B as a sql table in your database
stadium_data_df.to_sql('stadium_data',con,if_exists='replace')

3

In [16]:
# save the world cup match data set created in part B as a sql table in your database
# (you'll need to FIRST slice the data frame by selecting only the columns we want. Drop goals1 & goals2)

world_cup_df = world_cup_df.drop(columns=['goals1', 'goals2'])
world_cup_df.to_sql('worldcup',con,if_exists='replace')

64

In [17]:
# save the teams data set created in part B as a sql table in your database. Keep the following
# variables: "code","continent", & "name"
world_cup_teams_df = world_cup_teams_df[['code', 'continent', 'name']]
world_cup_teams_df.to_sql('worldcupteams',con,if_exists='replace')

32

In [18]:
# From the world cup match data table, use SQL syntax to select only those rows where Mexico played as team1 (team1.name) OR team2 (team2.name) from the database, and save these rows as a data frame
# HINT: You'll need to use WHERE to select the right rows
# VERY IMPORTANT HINT: if a column name includes a period (.), you must enclose the
# column name  AND the value for the WHERE statement in double quotes, and enclose the entire query in single quotes
# eg. 'SELECT * FROM df WHERE "col1.name"="Atlanta" OR "col2.name"="Atlanta"'


# count the number of rows in the data frame - This represents the number of games Mexico played in, as either team 1 or team 2

myquery = 'SELECT * FROM worldcup WHERE "team1.name" = "Mexico" OR "team2.name" = "Mexico"  '

word_cup_query = pd.read_sql_query(myquery,con)
print(word_cup_query.shape[0])
word_cup_query.head()

# print the first few rows of the data frame


4


,index,num,date,time,score1,score2,score1i,score2i,group,city,...,team1.code,team2.name,team2.code,stadium.key,stadium.name,score1et,score2et,score1p,score2p,knockout
0,10,11,2018-06-17,18:00,0,1,0,1,Group F,Moscow,...,GER,Mexico,MEX,luzhniki,Luzhniki Stadium,None,None,None,None,NaN
1,27,28,2018-06-23,18:00,1,2,0,1,Group F,Rostov-on-Don,...,KOR,Mexico,MEX,rostov,Rostov Arena,None,None,None,None,NaN
2,43,44,2018-06-27,19:00,0,3,0,0,Group F,Ekaterinburg,...,MEX,Sweden,SWE,ekaterinburg,Ekaterinburg Arena,None,None,None,None,NaN
3,52,53,2018-07-02,18:00,2,0,0,0,None,Samara,...,BRA,Mexico,MEX,samara,Samara Arena,None,None,None,None,1.0


In [19]:
# From the world cup match data table, select only those rows that were tied games from the database and save these rows as a data frame
# HINT: By tie, this means that score1 is the same as score2

myquery = 'SELECT * FROM worldcup WHERE score1 = score2  '

word_cup_query = pd.read_sql_query(myquery,con)
print(word_cup_query.shape[0])
word_cup_query.head()

# count the number of rows in the data frame


# print the first few rows of the data frame


14


,index,num,date,time,score1,score2,score1i,score2i,group,city,...,team1.code,team2.name,team2.code,stadium.key,stadium.name,score1et,score2et,score1p,score2p,knockout
0,2,3,2018-06-15,21:00,3,3,2,1,Group B,Sochi,...,POR,Spain,ESP,fisht,Fisht Stadium,NaN,NaN,NaN,NaN,NaN
1,6,7,2018-06-16,16:00,1,1,1,1,Group D,Moscow,...,ARG,Iceland,ISL,spartak,Spartak Stadium,NaN,NaN,NaN,NaN,NaN
2,8,9,2018-06-17,21:00,1,1,1,0,Group E,Rostov-on-Don,...,BRA,Switzerland,SUI,rostov,Rostov Arena,NaN,NaN,NaN,NaN,NaN
3,21,22,2018-06-21,16:00,1,1,1,1,Group C,Samara,...,DEN,Australia,AUS,samara,Samara Arena,NaN,NaN,NaN,NaN,NaN
4,31,32,2018-06-24,20:00,2,2,1,1,Group H,Ekaterinburg,...,JPN,Senegal,SEN,ekaterinburg,Ekaterinburg Arena,NaN,NaN,NaN,NaN,NaN
